In [1]:
import numpy as np
import scipy.stats as scp 

We have 

$$\left\lbrace\begin{array}{l}
d_1 = \frac{1}{\sigma\sqrt{\tau}}(ln(\frac{S}{K} + (r+\frac{\sigma^2}{2})\tau) \\[.1cm]
d_2= \frac{1}{\sigma\sqrt{\tau}}(ln(\frac{S}{K} + (r+\frac{\sigma^2}{2})\tau)
\end{array}\right.$$

with $\tau = T - t$

Noticed that $d_2 = d_1 - \sigma\sqrt{\tau}$

In [2]:
def d(S_, K_, r_,sigma_ ,tau_):
    d1_ = 1 / (sigma_ * np.sqrt(tau_)) * ( np.log(S_/K_) + (r_ + sigma_**2/2) * tau_)
    d2_ = d1_ - sigma_ * np.sqrt(tau_)
    return d1_, d2_

We have
$$\boxed{v(\sigma) = S\phi(d_1) - Ke^{-r\tau}\phi(d_2)}$$

with
$$\phi(x) = \frac{1}{\sqrt{2\pi}}\int_{-\infty}^{x}e^{-\frac{u^2}{2}}du$$

In [3]:
def call_option(S_, K_, r_,sigma_ ,tau_ ,d1_,d2_):
    return scp.norm.cdf(d1_) * S_ - scp.norm.cdf(d2_) * K_ * np.exp(-r_ *tau_)

We define the option's Vega by :
$$\boxed{g'(\sigma) = S\psi(d_1)\sqrt{\tau}}$$

where
$$\left\lbrace\begin{array}{l}
g(\sigma) = v(\sigma) - v^{mkt} \\[.1cm]
\psi(x) = \frac{1}{\sqrt{2\pi}}e^{-\frac{x^2}{2}}\\
d_1 = \frac{1}{\sigma\sqrt{\tau}}(ln(\frac{S}{K} + (r+\frac{\sigma^2}{2})\tau)
\end{array}\right.$$


In [4]:
def call_vega(S_, tau_ , d1_):
    return S_ * scp.norm.pdf(d1_) * np.sqrt(tau_)

Newton-Raphson's method :

$$\sigma_{n+1}=\sigma_n - \frac{g(\sigma)}{g'(\sigma)}$$

or

$$\boxed{\sigma_{n+1}= \sigma_n - \frac{v(\sigma)-v^{mkt}}{S\psi(d_1)}\sqrt{\tau}}$$

In [5]:
def Newton_Raphson(S_,K_,r_,tau_, sigma0_ ,price_, epsilon_):
    d1_,d2_ = d(S_, K_, r_,sigma0_ ,tau_)
    g = call_option(S_, K_, r_,sigma0_ ,tau_,d1_,d2_)-price_
    sigma_ = sigma0_ - g/call_vega(S_,tau_, d1_)
    while np.abs(sigma_-sigma0_)/sigma0_ > epsilon_:
        sigma0_ = sigma_
        d1_,d2_ = d(S_, K_, r_,sigma0_ ,tau_)
        g = call_option(S_, K_, r_,sigma0_ ,tau_,d1_,d2_)-price_
        sigma_ = sigma0_ - g/call_vega(S_,tau_, d1_)
    return sigma_


In [6]:
Newton_Raphson(194.11,210,0.01,60/365,0.5,1.50,1e-6)

0.20015597622419934